## Agent running on SQL Server
```markdown
make sure you have SQL Server with AdventureWorks database and it is accessible with SQL Auth

you need to set this en variable and fill in the params in the <>
SQLSERVER_CONNECTION_STRING  = "Driver={ODBC Driver 18 for SQL Server};Server=tcp:<YOUR SERVER NAME>.database.windows.net,1433;Database=<YOUR DATABASE NAME>;Uid=<YOUR SQL ADMIN USER>;Pwd=<PASSWORD>;Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;"

```


In [ ]:

import os
import asyncio
import json
import logging
from typing import List, Dict, Any, Optional, Annotated
import datetime
import nest_asyncio
import pyodbc
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.core.credentials import AccessToken
from semantic_kernel import Kernel
from semantic_kernel.agents import AgentGroupChat, ChatCompletionAgent
from semantic_kernel.agents.strategies import KernelFunctionSelectionStrategy, KernelFunctionTerminationStrategy
from semantic_kernel.functions.kernel_function_from_prompt import KernelFunctionFromPrompt
from semantic_kernel.functions import kernel_function, KernelArguments
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

load_dotenv()
nest_asyncio.apply()

kernel = Kernel()
kernel.add_service(AzureChatCompletion(service_id="sql-chat",
                                       deployment_name=os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME'),
                                       endpoint=os.getenv('AZURE_OPENAI_API_ENDPOINT'),
                                       api_key=os.getenv('AZURE_OPENAI_API_KEY'),
                                       api_version=os.getenv('AZURE_OPENAI_API_VERSION')))

In [ ]:
conn_str = os.getenv("SQLSERVER_CONNECTION_STRING")
if not conn_str:
    raise ValueError("SQLSERVER_CONNECTION_STRING environment variable is not set. Please set it with your SQL Server connection details.")


try:
    conn = pyodbc.connect(conn_str)
    conn.close()
    print("Successfully connected to SQL Server")
except Exception as e:
    print(f"Failed to connect to SQL Server: {str(e)}")
    raise

In [ ]:


class QuerySQLServerPlugin:
    def __init__(self, connection_string: str):
        self.connection_string = connection_string
        self.logger = logging.getLogger(__name__)
        self.logger.setLevel(logging.INFO)
        handler = logging.StreamHandler()
        formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
        handler.setFormatter(formatter)
        self.logger.addHandler(handler)

    @staticmethod
    def clean_sql(sql_query: str) -> str:
     
        cleaned = ' '.join(line.split('--')[0].strip() for line in sql_query.split('\n'))
        cleaned = ' '.join(cleaned.split())
        return cleaned

    def validate_query(self, sql_query: str) -> bool:
        
        dangerous_keywords = ['DROP', 'DELETE', 'UPDATE', 'INSERT', 'TRUNCATE', 'ALTER']
        query_upper = sql_query.upper()
        return not any(keyword in query_upper for keyword in dangerous_keywords)

    @kernel_function(
        name="query_sql_server",
        description="Executes a SQL query on SQL Server and returns formatted results."
    )
    async def query_sql_server(self, sql_query: str) -> str:
        def execute_query():
            self.logger.info(f"Executing query: {sql_query}")
            try:
                if not self.validate_query(sql_query):
                    return "Error: Query contains potentially dangerous operations"

                conn = pyodbc.connect(self.connection_string)
                cur = conn.cursor()
                query = self.clean_sql(sql_query)
                cur.execute(query)
                col_names = [desc[0] for desc in cur.description] if cur.description else []
                rows = cur.fetchall()
                cur.close()
                conn.close()
                
                if not rows:
                    return "No results found."
                
                results = [dict(zip(col_names, row)) for row in rows]
                self.logger.info(f"Query returned {len(results)} rows")
                return str(results)
            except Exception as e:
                self.logger.error(f"Query execution error: {str(e)}")
                return f"Query error: {str(e)}"
        return await asyncio.to_thread(execute_query)

    @kernel_function(
        name="get_table_schema",
        description="Retrieves the schema for specified SQL Server tables."
    )
    async def get_table_schema(self, table_names: Annotated[str, "Comma-separated table names"]) -> str:
        def execute_schema_retrieval():
            try:
                # Parse table names and schemas
                tables = []
                for name in table_names.split(","):
                    name = name.strip()
                    if '.' in name:
                        schema, table = name.split('.')
                        tables.append((schema.strip(), table.strip().replace('[','').replace(']','')))
                    else:
                        tables.append(('dbo', name.strip().replace('[','').replace(']','')))

                # Build dynamic schema query
                schema_conditions = []
                params = []
                for schema, table in tables:
                    schema_conditions.append("(TABLE_SCHEMA = ? AND TABLE_NAME = ?)")
                    params.extend([schema, table])

                schema_query = f"""
                    SELECT TABLE_SCHEMA, TABLE_NAME, COLUMN_NAME, DATA_TYPE, ORDINAL_POSITION
                    FROM INFORMATION_SCHEMA.COLUMNS
                    WHERE {' OR '.join(schema_conditions)}
                    ORDER BY TABLE_SCHEMA, TABLE_NAME, ORDINAL_POSITION;
                """
                
                self.logger.info(f"Retrieving schema for tables: {tables}")
                
                conn = pyodbc.connect(self.connection_string)
                cur = conn.cursor()
                cur.execute(schema_query, params)
                rows = cur.fetchall()
                cur.close()
                conn.close()

                schema = {}
                for schema_name, table_name, col_name, data_type, _ in rows:
                    key = f"{schema_name}.{table_name}"
                    if key not in schema:
                        schema[key] = []
                    schema[key].append({
                        "column_name": col_name,
                        "data_type": data_type
                    })

                schema_json = json.dumps(schema, indent=2)
                self.logger.info(f"Successfully retrieved schema for {len(schema)} tables")
                return schema_json
            except Exception as e:
                self.logger.error(f"Schema retrieval error: {str(e)}")
                return f"Schema retrieval error: {str(e)}"

        return await asyncio.to_thread(execute_schema_retrieval)
    
query_plugin = QuerySQLServerPlugin(conn_str)  





In [ ]:
# a dummy plugin to get the current date and time
class DatetimePlugin:
    @kernel_function(description="Returns the current date and time in YYYY-MM-DD HH:MM format.")
    async def get_current_datetime(self) -> str:
        return datetime.datetime.now().strftime("%Y-%m-%d %H:%M")
    
datetime_plugin = DatetimePlugin()


date_time_now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M")

# SQL Assistant Agent using the SQL Server plugin
sql_agent = ChatCompletionAgent(
    kernel=kernel,
    name="SQLAssistantAgent",
    instructions=f"""
    You are a helpful assistant that retrieves data from a SQL Server database about products, product specifications, and bugs.
    YOU must retrieve the latest schema using the get_table_schema function from the provided plugin BEFORE constructing your SQL queries.

    You MUST follow these rules precisely:
      1. You must always use the available tables below to construct your SQL queries.
      2. YOU must always get only and only related tables that you think are used in the query from the user's question.
      3. Invoke 'get_table_schema' with relevant table names FIRST. After reviewing the schema, THEN create your SQL query.
      4. Retrieve ONLY the schemas for the necessary tables by invoking 'get_table_schema'. NEVER retrieve all tables' schemas, ONLY the required ones.
      5. NEVER output raw SQL statements to the user.
      6. ALWAYS convert user questions into SQL and directly invoke the provided plugin function query_sql_server.
      7. ONLY provide the exact returned result from the plugin.
      8. NEVER describe or show the SQL query you created to the user.
      9. NEVER guess or invent table names.
      10. Make sure your query always and always does not generate duplicate rows.
      11. The date and time now is {date_time_now}.
       
      Tables are:
         ["HumanResources.Department", "HumanResources.Employee",
          "HumanResources.EmployeeDepartmentHistory", "HumanResources.EmployeePayHistory", "HumanResources.JobCandidate",
          "HumanResources.Shift", "Person.Address", "Person.AddressType", "Person.BusinessEntity", "Person.BusinessEntityAddress",
          "Person.BusinessEntityContact", "Person.ContactType", "Person.CountryRegion", "Person.EmailAddress", "Person.Password",
          "Person.Person", "Person.PersonPhone", "Person.PhoneNumberType", "Person.StateProvince", "Production.BillOfMaterials",
          "Production.Culture", "Production.Document", "Production.Illustration", "Production.Location", "Production.Product",
          "Production.ProductCategory", "Production.ProductCostHistory", "Production.ProductDescription", "Production.ProductDocument",
          "Production.ProductInventory", "Production.ProductListPriceHistory", "Production.ProductModel", "Production.ProductModelIllustration",
          "Production.ProductModelProductDescriptionCulture", "Production.ProductPhoto", "Production.ProductProductPhoto",
          "Production.ProductReview", "Production.ProductSubcategory", "Production.ScrapReason", "Production.TransactionHistory",
          "Production.TransactionHistoryArchive", "Production.UnitMeasure", "Production.WorkOrder", "Production.WorkOrderRouting",
          "Purchasing.ProductVendor", "Purchasing.PurchaseOrderDetail", "Purchasing.PurchaseOrderHeader", "Purchasing.ShipMethod",
          "Purchasing.Vendor", "Sales.CountryRegionCurrency", "Sales.CreditCard", "Sales.Currency", "Sales.CurrencyRate", "Sales.Customer",
          "Sales.PersonCreditCard", "Sales.SalesOrderDetail", "Sales.SalesOrderHeader", "Sales.SalesOrderHeaderSalesReason",
          "Sales.SalesPerson", "Sales.SalesPersonQuotaHistory", "Sales.SalesReason", "Sales.SalesTaxRate", "Sales.SalesTerritory",
          "Sales.SalesTerritoryHistory", "Sales.ShoppingCartItem", "Sales.SpecialOffer", "Sales.SpecialOfferProduct", "Sales.Store"]

      Examples:
      Which salespersons have sold more than $500,000 worth of products in the year 2013,
        along with their territory, total sales amount, top-selling product, and the number of orders they processed?
        generated query:
            WITH SalesData AS (
                SELECT 
                    soh.SalesPersonID,
                    sod.ProductID,
                    SUM(sod.LineTotal) AS TotalSales,
                    COUNT(DISTINCT soh.SalesOrderID) AS OrderCount
                FROM Sales.SalesOrderHeader soh
                JOIN Sales.SalesOrderDetail sod 
                    ON soh.SalesOrderID = sod.SalesOrderID
                WHERE YEAR(soh.OrderDate) = 2013
                GROUP BY soh.SalesPersonID, sod.ProductID
            ),
            RankedSales AS (
                SELECT 
                    SalesPersonID,
                    ProductID,
                    SUM(TotalSales) AS TotalSalesAmount,
                    SUM(OrderCount) AS TotalOrders,
                    ROW_NUMBER() OVER (
                    PARTITION BY SalesPersonID
                    ORDER BY SUM(TotalSales) DESC
                    ) AS ProductRank
                FROM SalesData
                GROUP BY SalesPersonID, ProductID
            )
            SELECT DISTINCT
                sp.BusinessEntityID AS SalesPersonID,
                p.FirstName + ' ' + p.LastName AS SalesPersonName,
                st.Name AS Territory,
                rs.TotalSalesAmount,
                pr.Name AS TopSellingProduct,
                rs.TotalOrders
            FROM RankedSales rs
            JOIN Sales.SalesPerson sp 
                ON rs.SalesPersonID = sp.BusinessEntityID
            JOIN Person.Person p 
                ON sp.BusinessEntityID = p.BusinessEntityID
            JOIN Sales.SalesTerritory st 
                ON sp.TerritoryID = st.TerritoryID
            JOIN Production.Product pr 
                ON rs.ProductID = pr.ProductID
            WHERE rs.ProductRank = 1
            AND rs.TotalSalesAmount > 50000
ORDER BY rs.TotalSalesAmount DESC;


results must be in a table format similar to below:
     SalesPersonID SalesPersonName          Territory  TotalSalesAmount   TopSellijngProduct        TotalOrders
     290	       RanjitVarkey Chudukatil	France	   152196.129776	  Mountain-200 Black, 38	1984
     279	       Tsvi Reiter	            Southeast  135082.061376	  Mountain-200 Black, 38	2174
     279	       Tsvi Reiter	            Southeast  135082.061376	  Mountain-200 Black, 38	2174
    """,
    plugins=[query_plugin]
)

# Datetime Assistant Agent
datetime_agent = ChatCompletionAgent(
    kernel=kernel,
    name='DatetimeAssistantAgent',
    instructions="""
    You are a helpful assistant that provides the current date and time.
    Always use the DatetimePlugin to fetch the latest datetime.
    Format your response exactly as 'The current date and time is: YYYY-MM-DD HH:MM'.
    """,
    plugins=[datetime_plugin]
)


In [ ]:
selection_function = KernelFunctionFromPrompt(
    function_name="selection",
    prompt="""
    Based on user query, respond with ONLY the exact agent name (no extra text):

    - date/time queries: DatetimeAssistantAgent
    - table name or metadata queries: SQLAssistantAgent
    - Queries in regards to Bugs: SQLAssistantAgent

    Rules:
        1. If the last message contains table names in a Python list format, use SQLAssistantAgent
        2. If the last message is from SQLAssistantAgent, terminate
        3. For date/time queries: DatetimeAssistantAgent
    User query:
    {{$lastmessage}}
    """,
)

termination_function = KernelFunctionFromPrompt(
    function_name="termination",
    prompt="""
    Determine if the conversation should terminate. Respond with ONLY 'yes' or 'no'.

    Terminate (respond with 'yes') if:
    1. The last message is from SQLAssistantAgent
    2. An error occurred
    3. The query has been fully answered

    Do not terminate (respond with 'no') if:
    1. We still need to process the query
    2. We're waiting for SQLAssistantAgent to execute the query
    3. We need more information

    Last message:
    {{$lastmessage}}
    """
)


In [ ]:
group_chat = AgentGroupChat(
    agents=[datetime_agent,  sql_agent],
    selection_strategy=KernelFunctionSelectionStrategy(
        function=selection_function,
        kernel=kernel,
        result_parser=lambda result: str(result.value[0]).strip(),
        history_variable_name="lastmessage",
    ),
    termination_strategy=KernelFunctionTerminationStrategy(
        function=termination_function,
        kernel=kernel,
        result_parser=lambda result: "yes" in str(result.value[0]).lower(),
        history_variable_name="lastmessage",
        maximum_iterations=3  # Allow up to 3 iterations for the agents to collaborate
    )
)

In [ ]:
import nest_asyncio
nest_asyncio.apply()

async def chat_with_agent(user_query):
    group_chat.is_complete = False
    await group_chat.add_chat_message(user_query)
    print(f"\nUser: {user_query}\n")

    try:
        async for response in group_chat.invoke():
            print(f"{response.role} ({response.name}): {response.content}")
    except Exception as e:
        print(f"\nAgentChatException occurred:\n{type(e).__name__}: {e}")
        if hasattr(e, 'args') and len(e.args) > 0:
            print("\nDetailed Error Info from Azure:")
            print(e.args)


#### Sample Questions:

- ist top 10 employees who are currently salaried and make sure the the employees are unique
- Which product has the highest total quantity sold in North America?
- Find all customers who have not placed any orders. 
- List the top 5 employees who have worked in the most departments over time, ordered by how many departments they’ve worked in.
- How many years from now is the last record in the sales table? 
- Which salespersons have sold more than $50,000 worth of products in the year 2013, along with their territory, total sales amount, top-selling product, and the number of orders they processed?  
- Which employees processed the highest number of high-value orders (over $10,000) in 2013, including their department, job title, and total orders handled?  


#### Make sure the SQL Auth is enabled before executing the next cell

In [ ]:
asyncio.run(chat_with_agent("How many years from now is the last record in the sales table?"))
